<a href="https://colab.research.google.com/github/YMGYM/google_colab_files/blob/master/Music_making_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro
this is my practice for [this post](https://tykimos.github.io/2018/09/14/How_to_Generate_Music_using_a_LSTM_Neural_Network_in_Keras/)

In [2]:
!git clone https://github.com/Skuldur/Classical-Piano-Composer.git /content/drive/My\ Drive/Datasets/FFsong

Cloning into '/content/drive/My Drive/Datasets/FFsong'...
remote: Enumerating objects: 334, done.
remote: Total 334 (delta 0), reused 0 (delta 0), pack-reused 334
Receiving objects: 100% (334/334), 721.79 MiB | 17.69 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Checking out files: 100% (101/101), done.


# Preparing Data

In [34]:
from music21 import converter, instrument, note, chord
import glob # 파일들의 리스트를 뽑는 라이브러리
import numpy as np

In [27]:
notes = []
for file in glob.glob('/content/drive/My Drive/Datasets/FFsong/midi_songs/*.mid'):
  midi = converter.parse(file)
  notes_to_parse = None
  try:
    parts = instrument.partitionByInstrument(midi) # 각각의 악기별로 파티션을 나누고, 다른 파트들들 하나로 합쳐줌
  except TypeError:
    print('## 1 {} file occur error'.format(file))
  if parts:
    print('## 2 {} file has instaument parts'.format(file))
    notes_to_parse = parts.parts[0].recurse() # 값들의 리스트를 반복할 수 있는 iterator 리턴
  else:
    print('## 3 {} file has notes in a flat structure'.format(file))
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    if isinstance(element, note.Note): # element 가 <music21.note.Note E-> 74.0 형태
      notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord): # element 가 <music21.chord.Chord F3 A3> 79.0 형태
      notes.append('.'.join(str(n) for n in element.normalOrder))

## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/0fithos.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/8.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/AT.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/BlueStone_LastDungeon.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/Cids.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/DOS.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/Eternal_Harvest.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/EyesOnMePiano.mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/FF3_Battle_(Piano).mid file has instaument parts
## 2 /content/drive/My Drive/Datasets/FFsong/midi_songs/FF3_Third_Phase_Final_(Piano).mid file has instaument parts
## 2 /content/dri

In [28]:
len(notes)

57177

# preprocessing

In [29]:
sequence_length = 100
# 모든 계이름의 이름을 pitchnames 변수에 저장함
# set: 중복 피함, sorted:정렬
pitchnames = sorted(set(item for item in notes))

# 계이름을 숫자로 바꾸는 딕셔너리 생성
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []

In [30]:
for i in range(0, len(notes) - sequence_length, 1):
  sequence_in = notes[i:i+ sequence_length]
  sequence_out = notes[i+sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

In [35]:
# 데이터 입력 형태를 LTSM에 맞게 변환
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# 입력값 정규화
network_input = network_input / float(n_vocab)
network_output = np.utils.to_categorical(network_output)

NameError: ignored